In [1]:
#!pip install torch torchvision
#!pip install transformers
#!pip install InstructorEmbedding
#!pip install sentence-transformers

In [ ]:
from InstructorEmbedding import INSTRUCTOR
import numpy as np
import re

model = INSTRUCTOR('hkunlp/instructor-xl')

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

#c = 0

def create_embeddings(doc):
    model.to("cuda")

    # Tokenize the document into sentences
    #sentences = doc.split(".")
    #sentences = re.split('(?<=[\.\?\!])\s*', doc)
    #sentences = [s.strip() for s in sentences if len(s.strip()) > 0]
    sentences = doc.split('\n')
    sentences = [sent_tokenize(sent) for sent in sentences if sent]
    sentences = [item for sublist in sentences for item in sublist]
    sentences = [s for s in sentences if len(s) > 10]
    #print(len(sentences))

    # Encode the sentences using SBERT
    instruction = "Represent a sentence for argumentation quality classification: "
    sentence_embeddings = []
    for sentence in sentences:
        embeddings = model.encode([[instruction,sentence]])
        sentence_embeddings.append(embeddings)
        #print(embeddings)

    # Compute the document embedding using hierarchical pooling
    #document_embedding = torch.stack(sentence_embeddings).max(dim=0).values
    document_embedding = np.mean(sentence_embeddings, axis=0).tolist()
    #print(type(document_embedding))
    #print(np.asarray(document_embedding).shape)
#    global c
#    c += 1
#    if c % 100 == 0:
#      print(c)
    # return the document embedding
    #print(document_embedding.shape)
    return document_embedding

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
import pandas as pd
#from tqdm import tqdm
#tqdm.pandas()

train_df = pd.read_csv("../../data/train-val-splits/qrels_args_docs_train.tsv", sep="\t")
train_df["instr_mean"] = train_df["prem"].apply(create_embeddings)

val_df = pd.read_csv("../../data/train-val-splits/qrels_args_docs_val.tsv", sep="\t")
val_df["instr_mean"] = val_df["prem"].apply(create_embeddings)

test_df = pd.read_csv("../../data/chatnoir_10_custom_stopw_lemmas.tsv", sep="\t")
test_df["instr_mean"] = test_df["html_plain"].apply(create_embeddings)

In [ ]:
train_df.to_csv("../../data/train-val-splits/qrels_args_docs_train_emb.tsv", sep="\t", index=False)
val_df.to_csv("../../data/train-val-splits/qrels_args_docs_val_emb.tsv", sep="\t", index=False)
test_df.to_csv("../../data/train-val-splits/chatnoir_10_custom_stopw_lemmas_emb.tsv", sep="\t", index=False)